🔀MODIFICACIÓN Y NORMALIZACIÓN DE LOS CSV DE EXTRACCIÓN

Tras las extracciones en el API de Spotify y cotejarlo con la información obtenida de Last.fm, consideramos necesarias ciertas modificaciones para que el volcado de estos en la base de datos transcurra de la mejor manera posible.

Sobre los CSV que almacenan la información sobre las canciones vamos a :
1. Eliminar la columna 'genre'❌
2. Renombrar la columna 'name_artist' a 'artist_name_raw'.🔄️
3. Añadir la columna 'artist_name_norm' que han de ser los nombres normalizados de los artistas.➕
4. Añadir la columna 'id_genre' que será un valor fijo para todas las filas del CSV de cada género (1 para pop, 2 para rock, 3 para chill y 4 para latin).➕

El resultado final de las columnas del Dataframe/CSV debe de ser: 'name_track' 'artist_name_raw', 'artist_name_norm', 'name_album', 'type', 'year', 'duration_ms', 'popularity' e 'id_genre'. Un total de 9 columnas.





In [44]:
import pandas as pd
import unicodedata
import re

In [ ]:
# Los CSV que habíamos generado anteriormente, los convertimos y guardamos a Dataframe.
df_rock_2019_2023 = pd.read_csv("Spotify_rock_2019_2023.csv")
df_pop_2019_2023 = pd.read_csv("Spotify_pop_2019_2023.csv")


In [ ]:
#Vamos a comprobar el estado actual de sus columnas.
df_rock_2019_2023.columns

Index(['name_track', 'name_artist', 'name_album', 'type', 'year',
       'duration_ms', 'genre', 'popularity'],
      dtype='object')

In [39]:
df_pop_2019_2023.columns

Index(['name_track', 'name_artist', 'name_album', 'type', 'year',
       'duration_ms', 'genre', 'popularity'],
      dtype='object')

In [ ]:
#---------------------------------------------------------------
# MODIFICACIONES GÉNERO ROCK
#---------------------------------------------------------------

# df_rock_2019_2023 = DataFrame original con:
# ['name_track','name_artist','name_album','type','year','duration_ms','genre','popularity']

# 1) Normalizar name_artist -> artist_name_norm
def normalizar_artista(nombre: str) -> str:
    if pd.isna(nombre):
        return ""
    s = str(nombre).strip().lower()

    # quitar acentos/diacríticos
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))

    # dejar letras/números/espacios
    s = re.sub(r"[^a-z0-9\s]", " ", s)

    # colapsar espacios
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_rock_2019_2023_def = df_rock_2019_2023.copy() #Hacemos una copia del DF original para evitar su pérdida
df_rock_2019_2023_def["artist_name_norm"] = df_rock_2019_2023_def["name_artist"].apply(normalizar_artista)

# 2) Eliminar 'genre'
df_rock_2019_2023_def = df_rock_2019_2023_def.drop(columns=["genre"])

# 3) Renombrar 'name_artist' -> 'artist_name_raw'
df_rock_2019_2023_def = df_rock_2019_2023_def.rename(columns={"name_artist": "artist_name_raw"})

# 4) Añadir 'id_genre' fijo = 2
df_rock_2019_2023_def["id_genre"] = 2

# 5) Reordenar columnas (incluyendo id_genre al final)
orden = [
    "name_track",
    "artist_name_raw",
    "artist_name_norm",
    "name_album",
    "type",
    "year",
    "duration_ms",
    "popularity",
    "id_genre"
]
df_rock_2019_2023_def = df_rock_2019_2023_def[orden]

df_rock_2019_2023_def.head()


,name_track,artist_name_raw,artist_name_norm,name_album,type,year,duration_ms,popularity,id_genre
0,Never There,Sum 41,sum 41,Order In Decline,track,2019,260093,54,2
1,Stand Out Fit In,ONE OK ROCK,one ok rock,Eye of the Storm,track,2019,214573,62,2
2,This Life,Vampire Weekend,vampire weekend,Father of the Bride,track,2019,268600,59,2
3,Seventeen,Sharon Van Etten,sharon van etten,Remind Me Tomorrow,track,2019,265920,61,2
4,MANTRA,Bring Me The Horizon,bring me the horizon,amo,track,2019,233128,62,2


In [46]:
df_rock_2019_2023_def.columns.tolist()


['name_track',
 'artist_name_raw',
 'artist_name_norm',
 'name_album',
 'type',
 'year',
 'duration_ms',
 'popularity',
 'id_genre']

In [ ]:
#Comprobamos que para todas las columnas de id_genre se le ha asignado el valor único 2
df_rock_2019_2023_def["id_genre"].unique()


array([2])

In [ ]:
# Como último paso, convertimos a CSV el Dataframe con las modificaciones del género rock.
df_rock_2019_2023_def.to_csv("Spotify_rock_final.csv", index=False, encoding="utf-8")


In [ ]:
#----------------------------------------------------------------
# MODIFICACIONES GÉNERO POP
#----------------------------------------------------------------

# df_pop_2019_2023 = DataFrame original con:
# ['name_track','name_artist','name_album','type','year','duration_ms','genre','popularity']

# 1) Normalizar name_artist -> artist_name_norm
def normalizar_artista(nombre: str) -> str:
    if pd.isna(nombre):
        return ""
    s = str(nombre).strip().lower()

    # quitar acentos/diacríticos
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))

    # dejar letras/números/espacios
    s = re.sub(r"[^a-z0-9\s]", " ", s)

    # colapsar espacios
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_pop_2019_2023_def = df_pop_2019_2023.copy() #Hacemos una copia del DF original para evitar su pérdida
df_pop_2019_2023_def["artist_name_norm"] = df_pop_2019_2023_def["name_artist"].apply(normalizar_artista)

# 2) Eliminar 'genre'
df_pop_2019_2023_def = df_pop_2019_2023_def.drop(columns=["genre"])

# 3) Renombrar 'name_artist' -> 'artist_name_raw'
df_pop_2019_2023_def = df_pop_2019_2023_def.rename(columns={"name_artist": "artist_name_raw"})

# 4) Añadir 'id_genre' fijo = 1
df_pop_2019_2023_def["id_genre"] = 1

# 5) Reordenar columnas (incluyendo id_genre al final)
orden = [
    "name_track",
    "artist_name_raw",
    "artist_name_norm",
    "name_album",
    "type",
    "year",
    "duration_ms",
    "popularity",
    "id_genre"
]
df_pop_2019_2023_def = df_pop_2019_2023_def[orden]

df_pop_2019_2023_def.head()


,name_track,artist_name_raw,artist_name_norm,name_album,type,year,duration_ms,popularity,id_genre
0,Breaking Me,Topic,topic,Breaking Me,track,2019,166793,73,1
1,Someone You Loved,Lewis Capaldi,lewis capaldi,Divinely Uninspired To A Hellish Extent,track,2019,182160,87,1
2,The Man,Taylor Swift,taylor swift,Lover,track,2019,190360,77,1
3,needy,Ariana Grande,ariana grande,"thank u, next",track,2019,171573,84,1
4,Higher Love,Kygo,kygo,Higher Love,track,2019,228267,73,1


In [54]:
df_pop_2019_2023_def.columns.tolist()


['name_track',
 'artist_name_raw',
 'artist_name_norm',
 'name_album',
 'type',
 'year',
 'duration_ms',
 'popularity',
 'id_genre']

In [55]:
# Como último paso, convertimos a CSV el Dataframe con las modificaciones del género pop.
df_pop_2019_2023_def.to_csv("Spotify_pop_final.csv", index=False, encoding="utf-8")


In [56]:
# Empecemos con la modificación de los dos CSV restantes.
#Primero los leemos y transformamos a Dataframe
df_chill_2019_2023 = pd.read_csv("Spotify_chill_2019_2023.csv")
df_latin_2019_2023 = pd.read_csv("Spotify_latin_2019_2023.csv")

In [57]:
df_chill_2019_2023.columns

Index(['name_track', 'name_artist', 'name_album', 'type', 'year',
       'duration_ms', 'genre', 'popularity'],
      dtype='object')

In [58]:
df_latin_2019_2023.columns

Index(['name_track', 'name_artist', 'name_album', 'type', 'year',
       'duration_ms', 'genre', 'popularity'],
      dtype='object')

In [ ]:
#----------------------------------------------------------------
# MODIFICACIONES GÉNERO CHILL
#----------------------------------------------------------------

# df_chill_2019_2023 = DataFrame original con:
# ['name_track','name_artist','name_album','type','year','duration_ms','genre','popularity']

# 1) Normalizar name_artist -> artist_name_norm
def normalizar_artista(nombre: str) -> str:
    if pd.isna(nombre):
        return ""
    s = str(nombre).strip().lower()

    # quitar acentos/diacríticos
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))

    # dejar letras/números/espacios
    s = re.sub(r"[^a-z0-9\s]", " ", s)

    # colapsar espacios
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_chill_2019_2023_def = df_chill_2019_2023.copy() #Hacemos una copia del DF original para evitar su pérdida
df_chill_2019_2023_def["artist_name_norm"] = df_chill_2019_2023_def["name_artist"].apply(normalizar_artista)

# 2) Eliminar 'genre'
df_chill_2019_2023_def = df_chill_2019_2023_def.drop(columns=["genre"])

# 3) Renombrar 'name_artist' -> 'artist_name_raw'
df_chill_2019_2023_def = df_chill_2019_2023_def.rename(columns={"name_artist": "artist_name_raw"})

# 4) Añadir 'id_genre' fijo = 3
df_chill_2019_2023_def["id_genre"] = 3

# 5) Reordenar columnas (incluyendo id_genre al final)
orden = [
    "name_track",
    "artist_name_raw",
    "artist_name_norm",
    "name_album",
    "type",
    "year",
    "duration_ms",
    "popularity",
    "id_genre"
]
df_chill_2019_2023_def = df_chill_2019_2023_def[orden]

df_chill_2019_2023_def.head()


,name_track,artist_name_raw,artist_name_norm,name_album,type,year,duration_ms,popularity,id_genre
0,Where We All Soar,Noah Jacob,noah jacob,Where We All Soar,track,2019,519383,0,3
1,Time 2 Quit - Remix Instrumental,BUPPON,buppon,enDroll : 12inch CLUB Package,track,2019,188223,0,3
2,Lotus Yoga,Sami Sivananda,sami sivananda,"Relax in Style, Vol. 3",track,2019,262789,0,3
3,The Breath Of Ganges,Mirage Of Deep,mirage of deep,"Chill Out Whisper, Vol. 2",track,2019,441493,0,3
4,Your Christmas Spirit,Tom Kalani,tom kalani,Atmospheric,track,2019,184390,0,3


In [61]:
# Como último paso, convertimos a CSV el Dataframe con las modificaciones del género pop.
df_chill_2019_2023_def.to_csv("Spotify_chill_final.csv", index=False, encoding="utf-8")


In [64]:
df_chill_2019_2023_def.shape

(667, 9)

In [ ]:
#----------------------------------------------------------------
# MODIFICACIONES GÉNERO LATIN
#----------------------------------------------------------------

# df_latin_2019_2023 = DataFrame original con:
# ['name_track','name_artist','name_album','type','year','duration_ms','genre','popularity']

# 1) Normalizar name_artist -> artist_name_norm
def normalizar_artista(nombre: str) -> str:
    if pd.isna(nombre):
        return ""
    s = str(nombre).strip().lower()

    # quitar acentos/diacríticos
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))

    # dejar letras/números/espacios
    s = re.sub(r"[^a-z0-9\s]", " ", s)

    # colapsar espacios
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_latin_2019_2023_def = df_latin_2019_2023.copy() #Hacemos una copia del DF original para evitar su pérdida
df_latin_2019_2023_def["artist_name_norm"] = df_latin_2019_2023_def["name_artist"].apply(normalizar_artista)

# 2) Eliminar 'genre'
df_latin_2019_2023_def = df_latin_2019_2023_def.drop(columns=["genre"])

# 3) Renombrar 'name_artist' -> 'artist_name_raw'
df_latin_2019_2023_def = df_latin_2019_2023_def.rename(columns={"name_artist": "artist_name_raw"})

# 4) Añadir 'id_genre' fijo = 4
df_latin_2019_2023_def["id_genre"] = 4

# 5) Reordenar columnas (incluyendo id_genre al final)
orden = [
    "name_track",
    "artist_name_raw",
    "artist_name_norm",
    "name_album",
    "type",
    "year",
    "duration_ms",
    "popularity",
    "id_genre"
]
df_latin_2019_2023_def = df_latin_2019_2023_def[orden]

df_latin_2019_2023_def.head()


,name_track,artist_name_raw,artist_name_norm,name_album,type,year,duration_ms,popularity,id_genre
0,Cristina,Sebastian Yatra,sebastian yatra,FANTASÍA,track,2019,201626,72,4
1,LA CANCIÓN,J Balvin,j balvin,OASIS,track,2019,242573,89,4
2,105 F Remix,KEVVO,kevvo,105 F Remix,track,2019,464200,68,4
3,Kemba Walker,Eladio Carrion,eladio carrion,Kemba Walker,track,2019,152960,66,4
4,La Playa,Myke Towers,myke towers,La Playa,track,2019,207731,67,4


In [65]:
df_latin_2019_2023_def.shape

(932, 9)

In [66]:
# Como último paso, convertimos a CSV el Dataframe con las modificaciones del género pop.
df_latin_2019_2023_def.to_csv("Spotify_latin_final.csv", index=False, encoding="utf-8")
